In [3]:
import tensorflow as tf
hello=tf.constant('hello')
sess=tf.Session()
print(sess.run(hello))

b'hello'


In [3]:
import tensorflow as tf
a=tf.constant(1)
b=tf.constant(2)
with tf.Session() as sess:
    print(sess.run(a+b))

3


In [5]:
sess=tf.InteractiveSession()#和seesion的区别
#tf.InteractiveSession()默认自己就是用户要操作的session，而tf.Session()没有这个默认，因此用eval()启动计算时需要指明session。
w1=tf.Variable(tf.random_normal([2,3],mean=1,stddev=1.0))
w2=tf.Variable(tf.random_normal([3,1],mean=1,stddev=1.0))
a=tf.constant([[0.7,0.9]])#注意两个
#初始化
tf.global_variables_initializer().run()
b=tf.matmul(a,w1)
y=tf.matmul(b,w2)
print(y.eval())#还要eval

[[1.4545964]]


In [7]:
tf.reset_default_graph()#v重置计算图
var1=tf.Variable(10.0,name="varname")
var2=tf.Variable(11.0,name="varname")
var3=tf.Variable(12.0)
var4=tf.Variable(13.0)
#定义get_variable变量
with tf.variable_scope("test1"):
    var5=tf.get_variable("varname",shape=[2],
                       dtype=tf.float32 )
with tf.variable_scope("test2"):
    var6=tf.get_variable("varname",shape=[2],
                       dtype=tf.float32 )
#输出变量
print("var1:",var1.name)
print("var2:",var2.name)
print("var3:",var3.name)
print("var4:",var4.name)
print("var5:",var5.name)
print("var6:",var6.name)

var1: varname:0
var2: varname_1:0
var3: Variable:0
var4: Variable_1:0
var5: test1/varname:0
var6: test2/varname:0


#### 每个节点是op  operation 

一个op获得多个tensor产生多个tensor
图必须在会话里被启动，会话将op发到CPU或gpu上，并提供执行operation的方法
分为构建阶段和执行阶段
构建图：
构建第一步，创建源  不需要任何输入
有默认图  一般只要一个图  用op构造器可以为其增加节点
op可以是常量
tf.   这类函数都是创建一个节点op包括tf.constant

为了真正进行运算 并且得到结果  就必须在会话里启动这个图
***

构造完后启动图  先创建一个session

变量需要被初始化
所以还要加一个初始化op

在调用run()之前并不会真正执行赋值操作
tf.Session.run()函数返回值为fetches的执行结果
eg... result = sess.run([mul,intermed])
result也是一个列表  同时去两个 这是fetch
run(
    fetches,
    feed_dict=None
    options=None,
    run_metadata=None
)返回的numpy类型的数据类型
tf.placeholder(dtype, shape=None, name=None)
为解决常量过多而添加了过多的节点，使用占位符，可以不马上赋值，在run那里feed_dict字典参数的形式赋值


In [15]:
tf.__path__

['C:\\Users\\Dell\\Anaconda3\\envs\\py36\\lib\\tensorflow']

In [9]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
plotdata = { "batchsize":[], "loss":[] } 
def moving_average(a, w=10): 
    if len(a) < w:       
        return a[:] 
    return [val if idx < w else sum(a[(idx-w):idx])/w for idx, val in enumerate(a)]   
train_X=np.linspace(-1,1,100)
train_Y=2*trainx+np.random.randn(*trainx.shape)*0.3
# plt.plot(trainx,trainy,'b',label="original data")
# plt.legend()
plt.show()
tf.reset_default_graph() 
#重置计算图
X = tf.placeholder("float") 
Y = tf.placeholder("float") 
W = tf.Variable(tf.random_normal([1]), name="weight") 
b = tf.Variable(tf.zeros([1]), name="bias") 
#前向结构
z = tf.multiply(X, W)+b
tf.summary.histogram('z',z)#将预测值以直方图显示
cost =tf.reduce_mean( tf.square(Y - z)) 
tf.summary.scalar('loss_function', cost) 
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
init = tf.global_variables_initializer() 
training_epochs = 20 
display_step = 2 
with tf.Session() as sess: 
    sess.run(init) 
    merged_summary_op = tf.summary.merge_all() 
    summary_writer = tf.summary.FileWriter('log/mnist_with_summaries',sess.graph)
    for epoch in range(training_epochs): 
        for (x, y) in zip(train_X, train_Y):
            sess.run(optimizer, feed_dict={X: x, Y: y})
            summary_str = sess.run(merged_summary_op,feed_dict={X: x, Y: y})
            summary_writer.add_summary(summary_str, epoch)
        if epoch % display_step == 0: 
            loss = sess.run(cost, feed_dict={X: train_X, Y:train_Y})
            print ("Epoch:", epoch+1, "cost=", loss,"W=", sess.run(W), "b=", sess.run(b))
            if not (loss == "NA" ): 
                plotdata["batchsize"].append(epoch)  
                plotdata["loss"].append(loss) 
    print (" Finished!") 
    print ("cost=", sess.run(cost, feed_dict={X: train_X, Y: train_Y}), "W=", sess.run(W), "b=", sess.run(b))   
    print ("cost:",cost.eval({X: train_X, Y: train_Y}))        
# plt.plot(train_X, train_Y, 'ro', label='Original data')    
# plt.plot(train_X, sess.run(W) * train_X + sess.run(b), label='Fitted line')     
# plt.legend()   
# plt.show()      
# plotdata["avgloss"] = moving_average(plotdata["loss"])    
# plt.figure(1)    
# plt.subplot(211)    
# plt.plot(plotdata["batchsize"], plotdata["avgloss"], 'b--')  
# plt.xlabel('Minibatch number')    
# plt.ylabel('Loss') 
# plt.title('Minibatch run vs. Training loss')       
# plt.show()
            
            


Epoch: 1 cost= 0.76823515 W= [0.72604233] b= [0.37894157]
Epoch: 3 cost= 0.13658093 W= [1.6582043] b= [0.12992042]
Epoch: 5 cost= 0.085958794 W= [1.9110562] b= [0.03471883]
Epoch: 7 cost= 0.08288456 W= [1.9766345] b= [0.00958421]
Epoch: 9 cost= 0.08277117 W= [1.9935949] b= [0.00307632]
Epoch: 11 cost= 0.082787454 W= [1.9979801] b= [0.0013935]
Epoch: 13 cost= 0.082794726 W= [1.9991137] b= [0.00095849]
Epoch: 15 cost= 0.08279681 W= [1.9994068] b= [0.00084596]
Epoch: 17 cost= 0.082797356 W= [1.9994829] b= [0.00081675]
Epoch: 19 cost= 0.082797505 W= [1.9995034] b= [0.00080894]
 Finished!
cost= 0.08279753 W= [1.9995065] b= [0.00080776]
cost: 0.08279753


In [3]:
import sys
sys.path

['',
 'C:\\Users\\Dell\\Anaconda3\\python36.zip',
 'C:\\Users\\Dell\\Anaconda3\\DLLs',
 'C:\\Users\\Dell\\Anaconda3\\lib',
 'C:\\Users\\Dell\\Anaconda3',
 'C:\\Users\\Dell\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\Dell\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\Dell\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Dell\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Dell\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Dell\\.ipython']

In [7]:
try:
    !jupyter nbconvert --to python 初学tensorflow.ipynb
except:
    pass

Active code page: 65001


[NbConvertApp] Converting notebook 初学tensorflow.ipynb to python
[NbConvertApp] Writing 4533 bytes to 初学tensorflow.py
